In [2]:
import os
import pandas as pd
import msoffcrypto
import tkinter as tk
from tkinter import filedialog, simpledialog

# Function to select files using a file dialog
def select_file(title):
    root = tk.Tk()
    root.withdraw()  # Hide the root window
    file_path = filedialog.askopenfilename(title=title, filetypes=[("Excel files", "*.xlsx")])
    return file_path

# Function to save the output file using a file dialog
def save_file(title):
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.asksaveasfilename(title=title, defaultextension=".xlsx", filetypes=[("Excel files", "*.xlsx")])
    return file_path

# Prompt user to select the input and output files
input_file = select_file("스마트스토어 다운로드 한 원본 엑셀파일 선택")
output_file = save_file("택배사 업로드 할 엑셀양식 저장하기")
output_folder = os.path.dirname(output_file)

# Prompt user to enter the password
root = tk.Tk()
root.withdraw()
password = simpledialog.askstring("Password Required", "엑셀 패스워드를 입력해주세요:", show='*')

if not password:
    print("패스워드를 입력해주세요!")
    exit()

decrypted_file_path = "decrypted_temp.xlsx"  # Temporary file for decrypted content

# Columns to extract
columns_to_extract = [
    "수취인명", 
    "수취인명", 
    "수취인연락처1", 
    "수취인연락처1", 
    "우편번호", 
    "통합배송지", 
    "수량", 
    "상품명", 
    "수량", 
    "최종 상품별 총 주문금액", 
    "옵션정보", 
    "수량", 
    "상품주문번호"
]
output_columns = [
    "수취고객명", 
    "수취인", 
    "수취인전화", 
    "수취인휴대폰", 
    "수취인우편번호",
    "수취인주소", 
    "총중량", 
    "상품명1", 
    "수량1", 
    "물품가격1",
    "MEMO", 
    "총수량", 
    "고객주문번호"
]

try:
    # Decrypt the input file
    with open(input_file, 'rb') as f:
        office_file = msoffcrypto.OfficeFile(f)
        office_file.load_key(password=password)

        with open(decrypted_file_path, 'wb') as decrypted_file:
            office_file.decrypt(decrypted_file)

    # Load the decrypted file
    df = pd.read_excel(decrypted_file_path, header=1)  # Skip the first row (header is on the second row)

    # Debug: Print available columns
    # print("Available columns in the input file:")
    # print(df.columns.tolist())

    # Extract specified columns
    extracted_data = pd.DataFrame()
    for src_col, dest_col in zip(columns_to_extract, output_columns):
        if src_col in df.columns:
            extracted_data[dest_col] = df[src_col]
        else:
            print(f"Warning: Column '{src_col}' not found in the input file.")

    # Save the extracted data to the output file
    extracted_data.to_excel(output_file, index=False, header=True)

    # Open the folder containing the output file
    if os.name == 'posix':
        os.system(f'xdg-open "{output_folder}"')
    else:
        os.startfile(output_folder)

    # print("Data has been successfully extracted and saved!")

except Exception as e:
    print(f"An error occurred: {e}")

finally:
    # Clean up the temporary decrypted file if it exists
    if os.path.exists(decrypted_file_path):
        os.remove(decrypted_file_path)


Available columns in the input file:
['상품주문번호', '주문번호', '배송속성', '상세배송지', '배송방법(구매자 요청)', '배송방법', '택배사', '송장번호', '풀필먼트사(주문 기준)', '발송일', '판매채널', '구매자명', '구매자ID', '수취인명', '주문상태', '주문세부상태', '수량클레임 여부', '결제위치', '결제일', '상품번호', '상품명', '상품종류', '반품안심케어', '옵션정보', '옵션관리코드', '수량', '옵션가격', '상품가격', '최종 상품별 할인액', '최초 상품별 할인액', '판매자 부담 할인액', '최종 상품별 총 주문금액', '최초 상품별 총 주문금액', '사은품', '발주확인일', '발송기한', '발송처리일', '송장출력일', '배송비 형태', '배송비 묶음번호', '배송비 유형', '배송비 합계', '제주/도서 추가배송비', '배송비 할인액', '판매자 상품코드', '판매자 내부코드1', '판매자 내부코드2', '수취인연락처1', '수취인연락처2', '통합배송지', '기본배송지', '구매자연락처', '우편번호', '배송메세지', '출고지', '결제수단', '네이버페이 주문관리 수수료', '매출연동 수수료', '정산예정금액', '매출연동수수료 유입경로', '개인통관고유부호', '주문일시', '배송희망일', '구독신청회차', '구독진행회차', '구독배송희망일']
Data has been successfully extracted and saved!
